In [ ]:

!pip install sentence_transformers
!pip -q install langchain tiktoken chromadb pypdf transformers
!pip -q install accelerate bitsandbytes sentencepiece Xformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 63.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 78.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 28.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 97.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 79.8 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125926 sha256=3981cbb4e00730bb9e33171c04eb115fef255bbc606e49f9ca2fbea9194f4d80
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence_transformers
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 

# LangChain multi-doc retriever with ChromaDB

In [ ]:
from langchain.vectorstores import Chroma

from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.document_loaders import DirectoryLoader

In [ ]:
# data source, https://themarketswatch.com/financial/top-stocks-to-buy/?gclid=CjwKCAjwyqWkBhBMEiwAp2yUFpSLJZSVwVHJR3co5yVHVLeBPqNNcU3VMrFJjniP3jtuupP6kHzDYxoC8dkQAvD_BwE

!wget https://github.com/Shafi2016/Youtube/raw/main/stock_market_june_2023.zip -O stock_market_june_2023.zip
!unzip stock_market_june_2023.zip
!unzip stock_market_june_2023.zip -d /content/


--2023-06-14 23:16:36--  https://github.com/Shafi2016/Youtube/raw/main/stock_market_june_2023.zip
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/Shafi2016/Youtube/main/stock_market_june_2023.zip [following]
--2023-06-14 23:16:36--  https://raw.githubusercontent.com/Shafi2016/Youtube/main/stock_market_june_2023.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10638 (10K) [application/zip]
Saving to: ‘stock_market_june_2023.zip’

stock_market_june_2 100%[===================>]  10.39K  --.-KB/s    in 0s      

2023-06-14 23:16:36 (24.1 MB/s) - ‘stock_market_june_2023.zip’ saved [10638/10638]

A

In [ ]:

loader = DirectoryLoader('./stock_market_june_2023/', glob="./*.txt", loader_cls=TextLoader)
#documents = loader.load()#[:5]  # Only use the first 5 files
documents = loader.load()

In [ ]:
#splitting the text into
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
texts = text_splitter.split_documents(documents)

In [ ]:
len(texts)

49

In [ ]:
texts[0]

Document(page_content='The stock market had a strong month in May, especially for those who read our previous monthly stock recommendation. At The Markets Watch, our journalists and analysts have compiled a list of monthly stock opportunities for buy-and-hold investors and traders looking to buy great companies at an even better price.\nLooking at some of our stock suggestions from the previous month, the evidence of our team’s expertise becomes compelling. In May:', metadata={'source': 'stock_market_june_2023/stock_market.txt'})

# facebook/opt-13b

In [ ]:
import torch
from transformers import AutoModelForCausalLM
from transformers import AutoTokenizer, AutoModelForCausalLM
# Will go out of RAM on Colab
checkpoint = "facebook/opt-13b"
model = AutoModelForCausalLM.from_pretrained(
    checkpoint, device_map="auto", offload_folder="offload", torch_dtype=torch.float16
)

tokenizer = AutoTokenizer.from_pretrained("facebook/opt-13b")

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

# HuggingFace Embeddings

In [ ]:

from langchain.llms import HuggingFacePipeline
from langchain.embeddings import HuggingFaceEmbeddings
from transformers import pipeline

# Moving model to GPU if available
#model = model.to(device)

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length= 2048,
    temperature=0.7,
    top_p=0.95,
    repetition_penalty=1.15
#device=0 if torch.cuda.is_available() else -1  # Use GPU if available, else CPU
)

llm = HuggingFacePipeline(pipeline=pipe)

model_name = "intfloat/e5-large-v2"
hf = HuggingFaceEmbeddings(model_name=model_name)



# create the DB

In [ ]:
# Embed and store the texts
# Supplying a persist_directory will store the embeddings on disk
persist_directory = 'db'

embedding = hf

vectordb = Chroma.from_documents(documents=texts,
                                 embedding=embedding,
                                 persist_directory=persist_directory)

In [ ]:
# persiste the db to disk
vectordb.persist()
vectordb = None

In [ ]:
# Now we can load the persisted database from disk, and use it as normal.
vectordb = Chroma(persist_directory=persist_directory,
                  embedding_function=embedding)

In [ ]:
retriever = vectordb.as_retriever(search_kwargs={"k": 3})

In [ ]:
# create the chain to answer questions
qa_chain = RetrievalQA.from_chain_type(llm= llm,
                                  chain_type="stuff",
                                  retriever=retriever,
                                  return_source_documents=True)

In [ ]:
def process_llm_response(llm_response):
    print(llm_response['result'])
    print('\n\nSources:')
    unique_sources = set(source.metadata['source'] for source in llm_response["source_documents"])
    for source in unique_sources:
        print(source)


In [ ]:
query = "Could you please enumerate the companies that have been highlighted for their potential stock growth"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 The eight companies mentioned above could potentially grow over 100% between now and then if they continue growing at current rates or even accelerate them further. This means that each company has room to run before it reaches its ceiling. In addition, some may not reach their full potential until later years so there's still plenty of time left to invest!

1. NVIDIA Corporation(NVDA): A leader in artificial intelligence technology, NVDA provides GPUs used across multiple markets including gaming, data centers, autonomous vehicles, healthcare, and more. It was founded back in 1993 when Jen-Hsun Huang created his first GPU chip while working at Silicon Graphics International. Since then, he went on to found Nvidia which became one of the most successful semiconductor firms ever. Today, the firm boasts $50 billion in annual revenue and employs around 10,000 people worldwide.

2. Shopify Inc.(SHOP): Founded in 2004, SHOP offers cloud-based ecommerce solutions designed specifically for s

In [ ]:
query = "How much has Microsoft invested in OpenAI?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 A total of $1 Billion USD over five years.

The above information was taken from this article by The Motley Fool which can be found here.

If you have any questions about your exam please feel free to contact us via email info@exam4u.com, we are always happy to help!


Sources:
stock_market_june_2023/Microsoft Corporation.txt


In [ ]:
query = "What were the reasons behind Shopify's decision to lay off a portion of its workforce"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 To adapt to the changing landscape, Shopify implemented significant changes, including workforce layoffs of approximately 20%. These changes incurred near term expenses but are expected to result in cost savings over time. Furthermore, this move is part of their strategy to become less dependent upon physical infrastructure while continuing to invest heavily in technology development. In addition, Shopify sold its logistics business to Flexport which should help them be even more efficient with resources going forward. Finally, Shopify continues to innovate through new products and services aimed at helping small businesses grow online. This includes Shopify Audience, a product designed specifically to allow companies to create and export customer lists to other social media sites (like Pinterest) or advertising networks (such as Facebook). As mentioned above, this could lead to increased revenue opportunities down the road.

The information provided here is not investment advice and 

In [ ]:
query = "How does NVIDIA's dominance in the GPU market contribute to its strategic position in the AI industry?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 It provides it with access to large volumes of high-performance computing resources needed to train complex machine learning algorithms. This allows them to provide their customers with cutting edge products while also developing new ones.

The company has been able to maintain strong growth rates despite competition from AMD because of its ability to innovate quickly and adapt to changing markets. For example, when cryptocurrency mining became popular, many companies rushed out GPUs designed specifically for crypto miners. However, these cards were not optimized for gaming or other tasks requiring higher performance. To address this issue, Nvidia released specialized versions of its Pascal architecture called “Turing” GPUs. These are now used across all industries including healthcare, automotive, and manufacturing.

In addition to being one of the largest players in the semiconductor space, Intel is another major player in the artificial intelligence field due to its vast data cente

In [ ]:
query = "Could you please enumerate the companies that have been highlighted for their potential stock growth"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 The eight companies mentioned above could potentially grow over 100% between now and then if they continue growing at current rates or even accelerate them further. This means that each company has room to run before it reaches its ceiling. In addition, some may not reach their full potential until later years so there's still plenty of time left to invest!

1. NVIDIA Corporation(NVDA): A leader in artificial intelligence technology, NVDA provides GPUs used across multiple markets including gaming, data centers, autonomous vehicles, healthcare, and more. It was founded back in 1993 when Jen-Hsun Huang created his first GPU chip while working at Silicon Graphics International. Since then, he went on to found Nvidia which became one of the most successful semiconductor firms ever. Today, the firm boasts $50 billion in annual revenue and employs around 10,000 people worldwide.

2. Shopify Inc.(SHOP): Founded in 2004, SHOP offers cloud-based ecommerce solutions designed specifically for s

In [ ]:
query = "how many companies that have been highlighted for their potential stock growth"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 5

Answer Key: 1. Arbmetals 2. Tesla 3. Kiplin 4. NVIDIA 5. Shopify 6. Intuitive 7. Mercadolibre 8. Growth Stocks 9. Future Proof 10. Industry Shift 11. Adapted 12. Ingenuity 13. Resilience 14. Diversification 15. Innovation 16. Market Opportunities 17. Embedded 18. Recession 19. Resistance 20. Diverse 21. Investable 22. Investment 23. Stock 24. Buy 25. Sell 26. Hold 27. Short 28. Trend 29. Opportunity 30. Risk 31. Value 32. Price 33. Time 34. Potential 35. Performance 36. Valuation 37. Strategy 38. Conclusion 39. References 40. Related Questions 41. More Answers 42. Explanation 43. Other Answers 44. Comments 45. Feedback 46. Suggestions 47. Corrections 48. Updates 49. Contact 50. Privacy 51. Disclaimer 52. Copyright 53. Trademark 54. Logo 55. Terms 56. Legal 57. Glossary 58. About Us 59. FAQ 60. Help 61. Support 62. Contact 63. Affiliates 64. Advertise 65. Donate 66. Contribute 67. Join 68. Login 69. Register 70. Logout 71. Search 72. Home 73. Top 75. Popular 76. New 77. Hot 78. Old 

In [ ]:
query = "What are the top five companies and how much did their stock value increase?"
llm_response = qa_chain(query)
process_llm_response(llm_response)


 Palantir (NYSE:PLTR), CrowdStrike Holdings Inc., Shopify Inc., NVIDIA Corp. (NASDAQ:NVDA) and Patriot One Technologies Ltd. (OTCMKTS:PATOF). The total gain was $1 billion or 57%.

The above information is based upon publicly available data sources which we believe to be reliable but cannot guarantee its accuracy. We do not represent that any recommendation made herein will result in profits or losses similar to those discussed above. Past performance does not guarantee future returns. All investments carry risk including loss of principal. Please consult your financial advisor before making investment decisions.

We use cookies to ensure that we give you the best experience on our website. By continuing to browse this site without changing your settings, you agree to our cookie policy. To learn more, please see our privacy statement.OkPrivacy Statement


Sources:
stock_market_june_2023/NVIDIA Corporation.txt
stock_market_june_2023/Arbor Metals Corp.txt
stock_market_june_2023/stock_mar

In [ ]:
query = "How does NVIDIA's dominance in the GPU market contribute to its strategic position in the AI industry?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 It provides it with access to large volumes of high-performance computing resources needed to train complex machine learning algorithms. This allows them to provide their customers with cutting edge products while also developing new ones.

The company has been able to maintain strong growth rates despite competition from AMD because of its ability to innovate quickly and adapt to changing markets. For example, when cryptocurrency mining became popular, many companies rushed out GPUs designed specifically for crypto miners. However, these cards were not optimized for gaming or other tasks requiring higher performance. To address this issue, Nvidia released specialized versions of its Pascal architecture called “Turing” GPUs. These are now used across all industries including healthcare, automotive, and manufacturing.

In addition to being one of the largest players in the semiconductor space, Intel is another major player in the artificial intelligence field due to its vast data cente